In [1]:
import os, sys
import random

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

In [ ]:
conf = SparkConf().setAppName('Explain_Min_Avg_Diff').setMaster('yarn') 
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [10]:
numItems = 100000
numParall = 5
rnds = [random.randint(0, numItems) for idx in range(numItems) ]
dfRnds = spark.sparkContext.parallelize(rnds, numParall).map(lambda x: (x, )).toDF(["rnd"])
spark.sql("drop table if exists tblRdns")
dfRnds.createOrReplaceTempView("tblRnds")
print("partitions num =", dfRnds.rdd.getNumPartitions())

partitions num = 5


In [11]:
spark.sql("select min(rnd) from tblRnds").explain(mode="extended")
spark.sql("select min(rnd) from tblRnds").show()

== Parsed Logical Plan ==
'Project [unresolvedalias('min('rnd), None)]
+- 'UnresolvedRelation [tblRnds], [], false

== Analyzed Logical Plan ==
min(rnd): bigint
Aggregate [min(rnd#39L) AS min(rnd)#42L]
+- SubqueryAlias tblrnds
   +- View (`tblRnds`, [rnd#39L])
      +- LogicalRDD [rnd#39L], false

== Optimized Logical Plan ==
Aggregate [min(rnd#39L) AS min(rnd)#42L]
+- LogicalRDD [rnd#39L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[min(rnd#39L)], output=[min(rnd)#42L])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=111]
      +- HashAggregate(keys=[], functions=[partial_min(rnd#39L)], output=[min#45L])
         +- Scan ExistingRDD[rnd#39L]

+--------+
|min(rnd)|
+--------+
|       0|
+--------+



In [12]:
spark.sql("select avg(rnd) from tblRnds").explain(mode="extended")
spark.sql("select avg(rnd) from tblRnds").show()

== Parsed Logical Plan ==
'Project [unresolvedalias('avg('rnd), None)]
+- 'UnresolvedRelation [tblRnds], [], false

== Analyzed Logical Plan ==
avg(rnd): double
Aggregate [avg(rnd#39L) AS avg(rnd)#58]
+- SubqueryAlias tblrnds
   +- View (`tblRnds`, [rnd#39L])
      +- LogicalRDD [rnd#39L], false

== Optimized Logical Plan ==
Aggregate [avg(rnd#39L) AS avg(rnd)#58]
+- LogicalRDD [rnd#39L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[avg(rnd#39L)], output=[avg(rnd)#58])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=151]
      +- HashAggregate(keys=[], functions=[partial_avg(rnd#39L)], output=[sum#62, count#63L])
         +- Scan ExistingRDD[rnd#39L]

+-----------+
|   avg(rnd)|
+-----------+
|50038.16595|
+-----------+



In [ ]:
# --

In [ ]:
_ = spark.sql("drop table if exists tblRdns")

In [ ]:
spark.stop()